In [17]:
# libraries
import pandas as pd
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np

# api
plotly.tools.set_credentials_file(username='acgarcia', api_key='asT7FaUCTFQ97M93vRLN')

# naming columns
cols = pd.read_csv('./stuff.csv')

# creating our dataframe
df_2017 = pd.read_csv('./personsx.csv', usecols = cols['Feature name in Doc'].str.strip())
df_2017 = df_2017.fillna(0)

In [18]:
# Subsetting and renaming
# dep stands for depression
dep_sex = df_2017.loc[:,['SEX','LACHRC17']]
dep_sex['SEX'] = dep_sex['SEX'].map({1:'male', 2:'female'})

In [19]:
# Cleaning our data into a desireable dataframe for both male and female
dep_male = pd.DataFrame(dep_sex[dep_sex['SEX'] == 'male']['LACHRC17'].value_counts())
dep_male = dep_male.reset_index()
dep_male = dep_male.loc[1:, :]
dep_male = dep_male.rename(index=str, columns={"index": "Code", "LACHRC17": "Male"})

dep_female = pd.DataFrame(dep_sex[dep_sex['SEX'] == 'female']['LACHRC17'].value_counts())
dep_female = dep_female.reset_index()
dep_female = dep_female.loc[1:, :]
dep_female = dep_female.rename(index=str, columns={"index": "Code", "LACHRC17": "Female"})

dep_sex = pd.concat([dep_male, dep_female], axis=1)
dep_sex = dep_sex.loc[:,~dep_sex.columns.duplicated()]
dep_sex['Code'] = dep_sex['Code'].map({1.0: 'Chronic', 2.0: 'Not chronic', 9.0:'Unknown if chronic'})

In [20]:
# Data for our bar plot
trace_male = go.Bar(
    x=dep_sex['Code'],
    y= dep_sex['Male'],
    name='Male'
)

trace_female = go.Bar(
    x=dep_sex['Code'],
    y= dep_sex['Female'],
    name='Female'
)

data = [trace_male, trace_female]
layout = go.Layout(
    barmode='stack',
    title = 'Depression/anxiety/emotional problem condition status among Males and Females'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')

Out of about 78k people, 921 females and 589 males are reported to have chronic depression. While for 9 females and 7 males, they did not know if it was chronic. 6 females and 2 males reported that it was not chronic. Lastly, the remaining people did not have an answer for this question.